In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mping
import glob
import os
import shutil

In [14]:
# Environment variable
main_data_dir = 'D:/Data/HAM100000 - Harvard Dataset'
metadata_dir = main_data_dir + '/HAM10000_metadata.csv'
img_data_dir = main_data_dir + '/img_data'
test_data_dir = main_data_dir + '/test'
test_label_dir = main_data_dir + '/test_label.csv'
preprocessed_data_dir = main_data_dir + '/preprocessed_data_20split'
train_dir = preprocessed_data_dir + '/train'
val_dir = preprocessed_data_dir + '/val'

train_label_path = preprocessed_data_dir + '/train_label.csv'
test_label_path = preprocessed_data_dir + '/val_label.csv'

In [15]:
data_pd = pd.read_csv(metadata_dir)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [16]:
data_pd = data_pd.dropna()
data_pd.shape

(9958, 8)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_df, test_df = train_test_split(data_pd, test_size=0.2, stratify=data_pd['dx'])

In [19]:
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'

#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,train


In [20]:
test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
1005,HAM_0007395,ISIC_0031133,bkl,consensus,70.0,female,trunk,vidir_molemax
3501,HAM_0000858,ISIC_0030944,nv,follow_up,35.0,male,trunk,vidir_molemax
2347,HAM_0007614,ISIC_0027672,vasc,histo,55.0,female,face,vidir_modern
413,HAM_0005514,ISIC_0024453,bkl,histo,65.0,female,face,rosendahl
9245,HAM_0001169,ISIC_0030384,nv,consensus,10.0,male,upper extremity,vidir_modern


In [21]:
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])
print("Number of train: {}".format(len(train_list)))
print("Number of test: {}".format(len(test_list)))

Number of train: 7966
Number of test: 1992


In [22]:
data_pd.set_index('image_id', inplace=True)

In [23]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [24]:
for i in targetnames:
  directory1=train_dir+'/'+i
  directory2=val_dir+'/'+i
  os.mkdir(directory1)
  os.mkdir(directory2)

In [25]:
for img in train_list:
    filename = img + '.jpg'
    label = data_pd.loc[img, 'dx']
    
    source = img_data_dir + '/' + filename
    
    target = os.path.join(train_dir, label, filename)
    
    shutil.copyfile(source, target)

In [26]:
for img in test_list:
    filename = img + '.jpg'
    label = data_pd.loc[img, 'dx']
    
    source = img_data_dir + '/' + filename
    target = os.path.join(val_dir, label, filename)
    
    shutil.copyfile(source, target)

In [27]:
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,train


In [28]:
drop_train = train_df.drop(['lesion_id', 'dx_type', 'dataset', 'train_test_split'], axis = 1)
drop_test = test_df.drop(['lesion_id', 'dx_type', 'dataset'], axis = 1)

In [29]:
drop_train.head()

,image_id,dx,age,sex,localization
0,ISIC_0027419,bkl,80.0,male,scalp
1,ISIC_0025030,bkl,80.0,male,scalp
2,ISIC_0026769,bkl,80.0,male,scalp
3,ISIC_0025661,bkl,80.0,male,scalp
4,ISIC_0031633,bkl,75.0,male,ear


In [30]:
drop_test.head()

,image_id,dx,age,sex,localization
1005,ISIC_0031133,bkl,70.0,female,trunk
3501,ISIC_0030944,nv,35.0,male,trunk
2347,ISIC_0027672,vasc,55.0,female,face
413,ISIC_0024453,bkl,65.0,female,face
9245,ISIC_0030384,nv,10.0,male,upper extremity


In [31]:
drop_train.to_csv(train_label_path)
drop_test.to_csv(test_label_path)